In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import zipfile

local_zip = 'drive/My Drive/datasets/cats-vs-dogs/train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [7]:
def create_standard_directories():
    root_dir = '/tmp/'
    subdirs = ['train_set/', 'validation_set/']
    for subdir in subdirs:
        label_dirs = ['cats/', 'dogs/']
        for label_dir in label_dirs:
            newdir = root_dir + subdir + label_dir
            os.makedirs(newdir, exist_ok=True)
            
create_standard_directories()

In [8]:
from shutil import move
from tqdm import tqdm

def move_images_into_directories():
    train_dir = '/tmp/train/'
    random.seed(1)
    validation_ratio = 0.25
    for img in tqdm(os.listdir(train_dir)):
        img_path = train_dir + img
        dst_dir = '/tmp/train_set/'
        if random.random() < validation_ratio:
            dst_dir = '/tmp/validation_set/'
            
        if img.startswith('cat'):
            dst = dst_dir + 'cats/' + img
            move(img_path, dst)
        
        elif img.startswith('dog'):
            dst = dst_dir + 'dogs/' + img
            move(img_path, dst)
        
        
move_images_into_directories()

100%|██████████| 25000/25000 [00:00<00:00, 36686.21it/s]


In [9]:
from tensorflow.keras.applications import VGG16

vgg_base = VGG16(weights='imagenet', 
                 include_top=False, 
                 input_shape=(150, 150, 3)
                 )

vgg_base.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [10]:
for layer in vgg_base.layers:
    layer.trainable = False
  
vgg_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [11]:
model = Sequential([vgg_base,
                    Flatten(),
                    Dropout(0.3),
                    Dense(512, activation='relu'),
                    Dropout(0.2),
                    Dense(1024, activation='relu'),
                    Dropout(0.1),
                    Dense(1, activation='sigmoid')
                    ])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                             horizontal_flip=True,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                            #  rotation_range=40,
                            #  shear_range=0.2,
                            #  fill_mode='nearest'
                                  )

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/tmp/train_set/',
                                              batch_size = 32,
                                              target_size = (150, 150),
                                              class_mode = 'binary'
                                              )

validation_generator = validation_datagen.flow_from_directory('/tmp/validation_set/',
                                                    batch_size = 32,
                                                    target_size = (150, 150),
                                                    class_mode = 'binary'
                                                    )

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


In [14]:
model.fit_generator(train_generator,
                    epochs=15,
                    validation_data=validation_generator
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
585/585 [==============================] - 172s 294ms/step - loss: 0.3748 - accuracy: 0.8290 - val_loss: 0.2542 - val_accuracy: 0.8908
Epoch 2/15
585/585 [==============================] - 167s 285ms/step - loss: 0.3079 - accuracy: 0.8625 - val_loss: 0.2447 - val_accuracy: 0.8910
Epoch 3/15
585/585 [==============================] - 168s 287ms/step - loss: 0.2986 - accuracy: 0.8700 - val_loss: 0.2221 - val_accuracy: 0.9050
Epoch 4/15
585/585 [==============================] - 164s 280ms/step - loss: 0.2901 - accuracy: 0.8717 - val_loss: 0.2314 - val_accuracy: 0.9075
Epoch 5/15
585/585 [==============================] - 162s 276ms/step - loss: 0.2832 - accuracy: 0.8769 - val_loss: 0.2228 - val_accuracy: 0.8978
Epoch 6/15
585/585 [==============================] - 162s 276ms/step - loss: 0.2725 - accuracy: 0.8818 - val_loss: 0.2081 - val_accuracy: 0.9100
Epoch 7/15
585/585 [============================

In [15]:
model.save('/content/drive/My Drive/datasets/cats-vs-dogs/VGG_model.h5')